In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import timedelta
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [51]:
USA = pd.read_csv('US_Accidents_Dec19.csv')

In [52]:
pd.set_option('display.max_columns', None)

In [53]:
USA.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [79]:
USA_2019['Distance(mi)'].value_counts()

0.00     202595
0.40         83
0.35         82
0.23         77
0.41         76
          ...  
4.82          1
9.26          1
13.07         1
11.92         1
10.73         1
Name: Distance(mi), Length: 892, dtype: int64

In [55]:
USA['Weather_Timestamp'] =  pd.to_datetime(USA['Weather_Timestamp'])
USA['Weather_Timestamp'].head()

0   2016-02-08 05:58:00
1   2016-02-08 05:51:00
2   2016-02-08 06:56:00
3   2016-02-08 07:38:00
4   2016-02-08 07:53:00
Name: Weather_Timestamp, dtype: datetime64[ns]

In [56]:
var = USA['Weather_Timestamp'].astype('str')

In [57]:
var.head()

0    2016-02-08 05:58:00
1    2016-02-08 05:51:00
2    2016-02-08 06:56:00
3    2016-02-08 07:38:00
4    2016-02-08 07:53:00
Name: Weather_Timestamp, dtype: object

In [58]:
var.str[:4]

0          2016
1          2016
2          2016
3          2016
4          2016
           ... 
2974330    2019
2974331    2019
2974332    2019
2974333    2019
2974334    2019
Name: Weather_Timestamp, Length: 2974335, dtype: object

In [59]:
USA['Date'] = var.str[:4]

In [60]:
USA.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Date'],
      dtype='object')

In [61]:
USA['Date']

0          2016
1          2016
2          2016
3          2016
4          2016
           ... 
2974330    2019
2974331    2019
2974332    2019
2974333    2019
2974334    2019
Name: Date, Length: 2974335, dtype: object

In [81]:
USA['Start_Time'] =  pd.to_datetime(USA['Start_Time'])
USA['Start_Time'].head()

0   2016-02-08 05:46:00
1   2016-02-08 06:07:59
2   2016-02-08 06:49:27
3   2016-02-08 07:23:34
4   2016-02-08 07:39:07
Name: Start_Time, dtype: datetime64[ns]

In [64]:
USA['End_Time'] =  pd.to_datetime(USA['End_Time'])
USA['End_Time'].head()

0   2016-02-08 11:00:00
1   2016-02-08 06:37:59
2   2016-02-08 07:19:27
3   2016-02-08 07:53:34
4   2016-02-08 08:09:07
Name: End_Time, dtype: datetime64[ns]

In [126]:
USA['Time_Lapse'] = (USA['End_Time'] - USA['Start_Time'])
USA['Time_Lapse'].head(10)

0   05:14:00
1   00:30:00
2   00:30:00
3   00:30:00
4   00:30:00
5   00:30:00
6   00:30:00
7   00:30:00
8   00:30:00
9   00:30:00
Name: Time_Lapse, dtype: timedelta64[ns]

In [128]:
USA['Time_Lapse'] = USA['Time_Lapse'].astype(int)
USA['Time_Lapse'].head(10)

0    18840000000000
1     1800000000000
2     1800000000000
3     1800000000000
4     1800000000000
5     1800000000000
6     1800000000000
7     1800000000000
8     1800000000000
9     1800000000000
Name: Time_Lapse, dtype: int64

In [129]:
USA['Time_Lapse'] = USA['Time_Lapse'].div(60000000000)

In [135]:
USA_2019 = USA[USA['Date'] == '2019']
USA['Time_Lapse'].isnull().sum()

0

In [136]:
USA_2019.shape

(941940, 51)

In [137]:
USA_2019 = USA_2019.drop(columns = ['End_Lat', 'End_Lng'], axis=1)

In [138]:
USA_2019.shape

(941940, 49)

In [139]:
USA_2019 = USA_2019.dropna()

In [140]:
USA_2019.shape

(212169, 49)

In [103]:
USA_2019['Severity'].to_frame()
USA_2019['Severity'].value_counts()

Series([], Name: Severity, dtype: int64)

In [141]:
target = USA_2019['Time_Lapse']

In [142]:
features = USA_2019[['Severity','Start_Lat','Start_Lng','Side','Temperature(F)','Wind_Chill(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction']]

In [143]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=34,test_size=0.2)

In [144]:
X_train.head()

,Severity,Start_Lat,Start_Lng,Side,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction
891273,2,35.030048,-82.220390,L,77.0,77.0,76.0,29.02,10.0,WSW
863864,2,34.742760,-80.812790,R,84.0,84.0,37.0,29.46,9.0,CALM
973891,2,38.014500,-84.537193,R,71.0,71.0,75.0,28.89,10.0,WNW
1124839,2,43.231659,-77.127861,L,19.9,7.3,85.0,29.94,0.8,NNE
577907,2,37.663151,-120.471649,L,57.0,57.0,63.0,29.63,10.0,NNW


In [145]:
model = sm.OLS(y_train,X_train.astype(float)).fit()
model.summary()

ValueError: could not convert string to float: 'L'